In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from telebot import types


In [ ]:


markup = types.ReplyKeyboardMarkup(row_width=2)
itembtn1 = types.KeyboardButton('a')
itembtn2 = types.KeyboardButton('v')

In [17]:
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
import requests
import telebot

# !nbdev_build_lib
from food.classify_image import *
token = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"

m = None

bot = telebot.TeleBot(token)
bot.dish = None
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Howdy, how are you doing?")

@bot.message_handler(content_types=['photo'])
def calssify_image(message):
	bot.send_chat_action(message.chat.id, 'typing') #does not seem to work
	global m
	m = message
	image_url = bot.get_file_url(message.photo[3].file_id)

	df = get_calories(url=image_url)
	df.index = df.index.str.replace('_100g','')
	bot.dish = df
	bot.dish['image_url'] = image_url
	bot.dish['user_id']= message.from_user.id

	r = df.to_frame('per 100g').to_string()

	# markup = types.ReplyKeyboardMarkup(row_width=2)
	# cancel = types.KeyboardButton('cancel')
	# markup.add(cancel)


	bot.reply_to(message, r)
	bot.reply_to(message, "specify weight of the dish in gramms to add it to the journal")


@bot.message_handler(regexp="[0-9]+")
def handle_grams(message):
	global m
	m = message
	if hasattr(bot,'dish'):  
		bot.dish['grams']=int(message.text)
		bot.dish['timestamp']=pd.Timestamp.utcnow() #should be user local time
	bot.reply_to(message, bot.dish.to_string())


bot.infinity_polling()

2022-03-30 23:06:55,919 (__init__.py:621 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-03-30 23:06:55,920 (__init__.py:623 MainThread) ERROR - TeleBot: "Break infinity polling"


In [20]:
bot.dish.to_frame().T

,energy_kcal,proteins,fat,carbohydrates,image_url,user_id,grams,timestamp
0,476.33,14.2,20.43,38.99,https://api.telegram.org/file/bot5091011572:AA...,78882798,150,2022-03-30 20:06:14.907667+00:00


In [15]:
message = m

In [16]:
int(message.text)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [14]:
bot.dish

energy_kcal                                                 476.33
proteins                                                      14.2
fat                                                          20.43
carbohydrates                                                38.99
image_url        https://api.telegram.org/file/bot5091011572:AA...
user_id                                                   78882798
dtype: object

In [89]:
bot.dish/100*int(message.text)

energy_kcal      38.266
proteins          0.106
fat               0.704
carbohydrates     6.614
grams             4.000
dtype: float64

In [88]:
bot.dish

energy_kcal      191.33
proteins           0.53
fat                3.52
carbohydrates     33.07
grams             20.00
dtype: float64

In [73]:
bot.dish/100*250

energy_kcal      151.675
proteins           4.400
fat                2.900
carbohydrates     13.850
dtype: float64

In [74]:
bot.dish

energy_kcal      60.67
proteins          1.76
fat               1.16
carbohydrates     5.54
dtype: float64

In [61]:
message.chat.id

78882798

In [30]:
file_url = bot.get_file_url(message.photo[3].file_id)

In [31]:
df = get_calories(url=file_url)

In [52]:
df.index = df.index.str.replace('_100g','')

In [53]:
t = df.to_frame('per 100g').to_string()

In [54]:
print(t)

               per 100g
energy_kcal      210.33
proteins           4.93
fat               10.47
carbohydrates     23.33


In [55]:
bot.reply_to(message, t)

In [17]:
file_info

'https://api.telegram.org/file/bot5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY/photos/file_89.jpg'

In [14]:
file_info.file_path

'photos/file_89.jpg'

In [15]:
f'https://api.telegram.org/file/bot{token}/{file_info.file_path}'

'https://api.telegram.org/file/bot5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY/photos/file_89.jpg'

In [10]:
m.chat.id

78882798